In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import sys, os, re, csv, codecs, numpy as np, pandas as pd
import time

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, concatenate
from keras.layers import Bidirectional, GlobalMaxPool1D, Convolution1D, MaxPooling1D
from keras.models import Model, Sequential
from keras.callbacks import TensorBoard

from nlp.glove import Glove
from database.utils import get_train_test_data
from evaluation.metrics import *
from models.nn_models import *

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [15]:
train_test_data = get_train_test_data()
X_train, y_train, X_test, y_test = get_train_test_data(merge=True)

glove = Glove.load()
w2v = glove.get_dict()

Found 1193514 word vectors.


In [16]:
embed_size = 200 # how big is each word vector
max_features = 20000 # dictionary size
maxlen = 75 # max number of words in a tweet to use

X_train, y_train, X_test, y_test = get_train_test_data(merge=True)

tokenizer = Tokenizer(num_words=max_features)

tokenizer.fit_on_texts(list(X_train))
list_tokenized_train = tokenizer.texts_to_sequences(X_train)
list_tokenized_test = tokenizer.texts_to_sequences(X_test)
X_train = pad_sequences(list_tokenized_train, maxlen)
X_test = pad_sequences(list_tokenized_test, maxlen)

embedding_matrix = glove.get_embedding_matrix(tokenizer, max_features, embed_size)
model = get_lstm_model(embedding_matrix)
#model = get_cnn_model(embedding_matrix)
model.summary()
ts = time.time()
model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          epochs=5,
          batch_size=256,
          callbacks=[RocAucEvaluation(validation_data=(X_test, y_test)),
                     TensorBoard(log_dir='./logs',
                                 histogram_freq=0,
                                 write_graph=True,
                                 write_images=False)])
te = time.time()
train_time = te - ts

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 75)                0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 75, 200)           3069000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 75, 100)           100400    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_3 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 153       
Total para

In [4]:
ts = time.time()
y_scores = model.predict(X_test)
te = time.time()
predict_time = te - ts
results = {}

In [22]:
measures, _, _, _ = class_report_multilabel(y_test, y_scores)
results['lstm'] = measures
results['lstm']['predict_time'] = predict_time
results['lstm']['train_time'] = train_time

pd.DataFrame(results).T

ROC AUC for class 0: 0.96
Precision for class 0: 0.72
ROC AUC for class 1: 0.88
Precision for class 1: 0.66
ROC AUC for class 2: 0.98
Precision for class 2: 0.69
Average precision score, micro-averaged over all classes: 0.67
Test score: 0.92



,accuracy,avg_precision,f1score,precision,predict_time,recall,specificity,test_roc_auc
lstm,0.746498,0.672772,0.641173,0.658377,3.921195,0.624845,0.955262,0.923353


In [5]:
measures, _, _, _ = class_report_multilabel(y_test, y_scores)
results['cnn'] = measures
results['cnn']['predict_time'] = predict_time
results['cnn']['train_time'] = train_time

pd.DataFrame(results).T

ROC AUC for class 0: 0.97
Precision for class 0: 0.77
ROC AUC for class 1: 0.87
Precision for class 1: 0.64
ROC AUC for class 2: 0.97
Precision for class 2: 0.75
Average precision score, micro-averaged over all classes: 0.70
Test score: 0.92



,accuracy,avg_precision,f1score,precision,predict_time,recall,specificity,test_roc_auc,train_time
cnn,0.765477,0.699341,0.630234,0.735099,5.067211,0.551553,0.972575,0.920489,238.946852


In [ ]:
for Xr_train, y_train, Xr_test, y_test, indicator in train_test_data:
    print(indicator)
    tokenizer = Tokenizer(num_words=max_features)
    
    tokenizer.fit_on_texts(list(Xr_train))
    list_tokenized_train = tokenizer.texts_to_sequences(Xr_train)
    list_tokenized_test = tokenizer.texts_to_sequences(Xr_test)
    X_train = pad_sequences(list_tokenized_train, maxlen)
    X_test = pad_sequences(list_tokenized_test, maxlen)
    
    #embedding_matrix = get_embedding_matrix(max_features, tokenizer, w2v, embed_size)
    y_pred = model.predict(X_test)
    
    
    #model = get_model(embedding_matrix)
    #model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, 
    #      callbacks=[roc_callback(training_data=(X_train,
    #                                             y_train),
    #                              validation_data=(X_test, y_test))])